In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
# โหลดข้อมูลจากไฟล์ CSV
df = pd.read_csv('UNSW_NB15_testing-set.csv')

In [2]:
X = df[['spkts', 'dpkts', 'sbytes', 'dbytes']]
y = df['label']

In [3]:
from sklearn.utils import resample

# รวม X และ y เข้าด้วยกัน
combined_data = pd.concat([X, y], axis=1)

# หาจำนวนตัวอย่างที่น้อยที่สุด
min_samples = combined_data['label'].value_counts().min()

# สร้างข้อมูลที่ undersampled จากแต่ละกลุ่ม
undersampled_data = combined_data.groupby('label').apply(lambda x: resample(x, n_samples=min_samples, random_state=42))

# แยก X และ y ออกจาก undersampled_data
X_resampled = undersampled_data[['spkts', 'dpkts', 'sbytes', 'dbytes']]
y_resampled = undersampled_data['label']


In [4]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# หา Information Gain ของแต่ละ feature
ig_scores = mutual_info_classif(X_resampled, y_resampled, discrete_features=False)

# แสดง Information Gain ของแต่ละ feature
for feature, ig_score in zip(X_resampled.columns, ig_scores):
    print(f'Information Gain for {feature}: {ig_score}')

# ใช้ DecisionTreeClassifier
model = DecisionTreeClassifier()

# วนซ้ำโดยใช้ค่า random seed แต่ละครั้ง
for seed in [1, 20, 100, 200, 1000]:
    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=seed)

    # ใช้ DecisionTreeClassifier
    model.fit(X_train, y_train)

    # ทำนายค่า 'label' ด้วยชุดทดสอบ
    y_pred = model.predict(X_test)

    # ประเมินประสิทธิภาพ
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f'Seed = {seed}, Accuracy: {accuracy}')

    # แสดง Confusion Matrix
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(conf_matrix)

    # แสดง Classification Report
    class_report = metrics.classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)


Information Gain for spkts: 0.19455403200517196
Information Gain for dpkts: 0.29625822576983607
Information Gain for sbytes: 0.520886015524805
Information Gain for dbytes: 0.39553316224482704
Seed = 1, Accuracy: 0.9427678571428572
Confusion Matrix:
[[10743   527]
 [  755 10375]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     11270
           1       0.95      0.93      0.94     11130

    accuracy                           0.94     22400
   macro avg       0.94      0.94      0.94     22400
weighted avg       0.94      0.94      0.94     22400

Seed = 20, Accuracy: 0.9421428571428572
Confusion Matrix:
[[10562   533]
 [  763 10542]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94     11095
           1       0.95      0.93      0.94     11305

    accuracy                           0.94     22400
   macro avg       0.94      0.94      0